In [53]:
import itertools
import numpy as np
import pandas as pd 
from numbers import Number
import sqlite3
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import statsmodels.api as sm
warnings.filterwarnings('ignore')

In [54]:
conn = sqlite3.connect('Data/im.db')


OperationalError: unable to open database file

In [ ]:
imdb_movie_ratings = pd.read_sql('''
SELECT *

FROM movie_ratings
    JOIN movie_basics
    USING (movie_id)

''', conn)
imdb_movie_ratings = imdb_movie_ratings.drop(['movie_id', 'averagerating', 'numvotes', 'start_year'], axis = 1)

In [ ]:
imdb_movie_ratings = imdb_movie_ratings[(imdb_movie_ratings['original_title'].isin(titles))| (imdb_movie_ratings['primary_title'].isin(titles))]

imdb_movie_ratings



In [ ]:
people = pd.read_sql('''
SELECT p.person_id, p.category, mb.primary_title, pe.primary_name

FROM principals as p
    JOIN movie_basics as mb
    USING(movie_id)
    JOIN persons as pe
    USING (person_id)
    
''', conn)



In [ ]:
reg_lang = pd.read_sql('''
SELECT mak.language, mak.region, mb.primary_title

FROM movie_akas as mak
    JOIN movie_basics as mb
    USING(movie_id)
    
    
''', conn)
language_dict = {
    'bg': 'Bulgarian',
    'he': 'Hebrew',
    'fr': 'French',
    'tr': 'Turkish',
    'ka': 'Georgian',
    'ta': 'Tamil',
    'te': 'Telugu',
    'en': 'English',
    'sv': 'Swedish',
    'hr': 'Croatian',
    'sl': 'Slovenian',
    'ca': 'Catalan',
    'fa': 'Persian',
    'es': 'Spanish',
    'cs': 'Czech',
    'nl': 'Dutch',
    'sr': 'Serbian',
    'sk': 'Slovak',
    'et': 'Estonian',
    'it': 'Italian',
    'hu': 'Hungarian',
    'da': 'Danish',
    'de': 'German',
    'el': 'Greek',
    'pt': 'Portuguese',
    'fi': 'Finnish',
    'no': 'Norwegian',
    'ru': 'Russian',
    'uk': 'Ukrainian',
    'ro': 'Romanian',
    'lt': 'Lithuanian',
    'cmn': 'Mandarin Chinese',
    'bs': 'Bosnian',
    'hi': 'Hindi',
    'th': 'Thai',
    'ms': 'Malay',
    'ar': 'Arabic',
    'qbp': None,  # You have 'qbp' with no corresponding full name.
    'ml': 'Malayalam',
    'yue': 'Cantonese',
    'qbn': None,  # You have 'qbn' with no corresponding full name.
    'id': 'Indonesian',
    'gl': 'Galician',
    'ga': 'Irish',
    'mr': 'Marathi',
    'pl': 'Polish',
    'tl': 'Tagalog',
    'vi': 'Vietnamese',
    'eu': 'Basque',
    'ja': 'Japanese',
    'bn': 'Bengali',
    'ur': 'Urdu'
}
reg_lang = reg_lang.replace(({"language": language_dict}))

reg_lang_money = pd.merge(reg_lang, budget_gross_cleaned, left_on = 'primary_title', right_on = 'title')
reg_lang_money

In [ ]:
people = people[people['primary_title'].isin(titles)]
people.primary_title.value_counts()

In [ ]:
imdb_ratings_money = pd.merge(imdb_movie_ratings, budget_gross_cleaned, left_on = 'primary_title', right_on = 'title')
imdb_ratings_money = imdb_ratings_money.drop(['primary_title', 'original_title', 'studio', 'year', 'id', 'release_date'], axis = 1)

In [ ]:
people

In [ ]:
low_budget_ratings = imdb_ratings_money[imdb_ratings_money['production_budget'] < 5000000]
low_budget_ratings_moneymaker = low_budget_ratings[low_budget_ratings['WW_success'] > 1]


In [ ]:


mid_budget_ratings = imdb_ratings_money[(imdb_ratings_money['production_budget'] > 5000000) & (imdb_ratings_money['production_budget'] < 50000000)]
mid_budget_ratings_moneymaker = mid_budget_ratings[mid_budget_ratings['WW_success'] > 1]

In [ ]:
high_budget_ratings = imdb_ratings_money[imdb_ratings_money['production_budget'] > 50000000]
high_budget_ratings_moneymaker = high_budget_ratings[high_budget_ratings['WW_success'] > 1]

In [ ]:
low_budget_no_outlier = low_budget[low_budget['WW_success'] < 1400]

In [ ]:
low_budget_no_outlier.describe().iloc[:,-3:]


In [ ]:
#boxplot = low_budget_no_outlier.boxplot(column=['domestic_success', 'foreign_success', 'WW_success'])
fig, ax = plt.subplots(1,3, figsize = (12,8), sharey= True)

ax[0].boxplot(low_budget_no_outlier['domestic_success'], showfliers=False)

ax[0].set_title('Low Budget Domestic Success')
ax[0].set_ylabel('Domestic Success')

ax[1].boxplot(low_budget_no_outlier['foreign_success'],showfliers=False)
ax[1].set_title('Low Budget Foreign Success')
ax[1].set_ylabel('Foreign Success')

ax[2].boxplot(low_budget_no_outlier['WW_success'],showfliers=False)
ax[2].set_title('Low Budget Worldwide Success')
ax[2].set_ylabel('Worldwide Success')

In [ ]:
mid_budget.describe().iloc[:,-3:]

In [ ]:
#boxplot = mid_budget.boxplot(column=['domestic_success', 'foreign_success', 'WW_success'])

fig, ax = plt.subplots(1,3, figsize = (12,8), sharey= True)

ax[0].boxplot(mid_budget['domestic_success'])

ax[0].set_title('Mid Budget Domestic Success')
ax[0].set_ylabel('Domestic Success')

ax[1].boxplot(mid_budget['foreign_success'])
ax[1].set_title('Mid Budget Foreign Success')
ax[1].set_ylabel('Foreign Success')

ax[2].boxplot(mid_budget['WW_success'])
ax[2].set_title('Mid Budget Worldwide Success')
ax[2].set_ylabel('Worldwide Success')


In [ ]:
high_budget.describe().iloc[:,-3:]

In [ ]:
#boxplot = high_budget.boxplot(column=['domestic_success', 'foreign_success', 'WW_success'])

fig, ax = plt.subplots(1,3, figsize = (12,8), sharey= True)

ax[0].boxplot(high_budget['domestic_success'])

ax[0].set_title('High Budget Domestic Success')
ax[0].set_ylabel('Domestic Success')

ax[1].boxplot(high_budget['foreign_success'])
ax[1].set_title('High Budget Foreign Success')
ax[1].set_ylabel('Foreign Success')

ax[2].boxplot(high_budget['WW_success'])
ax[2].set_title('High Budget Worldwide Success')
ax[2].set_ylabel('Worldwide Success')


In [ ]:
fig, ax = plt.subplots(1,3, figsize = (20,8), sharex = True)

ax[0].hist(low_budget_ratings_moneymaker['runtime_minutes'], bins = 20)


ax[1].hist(mid_budget_ratings_moneymaker['runtime_minutes'],bins = 20)
ax[2].hist(high_budget_ratings_moneymaker['runtime_minutes'],bins = 20)

ax[0].set_title('Low Budget Runtime Distribution')
ax[0].set_xlabel('Runtime (minutes)')
ax[1].set_title('Mid Budget Runtime Distribution')
ax[1].set_xlabel('Runtime (minutes)')
ax[2].set_title('High Budget Runtime Distribution')
ax[2].set_xlabel('Runtime (minutes)')

In [ ]:
x_bar = np.mean(low_budget_ratings_moneymaker['runtime_minutes'])
s = np.std(low_budget_ratings_moneymaker['runtime_minutes'], ddof = 1)
se = s/np.sqrt(len(low_budget_ratings_moneymaker['runtime_minutes']))
print('Sample mean:', x_bar)
print('Sample standard deviation:', s)
print('Estimated standard error:', se)
stats.t.interval(confidence = 0.95,                              
                 df= len(low_budget_ratings_moneymaker['runtime_minutes'])-1,             
                 loc = x_bar,                               
                 scale = se)    

In [ ]:
x_bar = np.mean(mid_budget_ratings_moneymaker['runtime_minutes'])
s = np.std(mid_budget_ratings_moneymaker['runtime_minutes'], ddof = 1)
se = s/np.sqrt(len(mid_budget_ratings_moneymaker['runtime_minutes']))
print('Sample mean:', x_bar)
print('Sample standard deviation:', s)
print('Estimated standard error:', se)
stats.t.interval(confidence = 0.95,                              
                 df= len(mid_budget_ratings_moneymaker['runtime_minutes'])-1,             
                 loc = x_bar,                               
                 scale = se)    

In [ ]:
x_bar = np.mean(high_budget_ratings_moneymaker['runtime_minutes'])
s = np.std(high_budget_ratings_moneymaker['runtime_minutes'], ddof = 1)
se = s/np.sqrt(len(high_budget_ratings_moneymaker['runtime_minutes']))
print('Sample mean:', x_bar)
print('Sample standard deviation:', s)
print('Estimated standard error:', se)
stats.t.interval(confidence = 0.95,                              
                 df= len(high_budget_ratings_moneymaker['runtime_minutes'])-1,             
                 loc = x_bar,                               
                 scale = se)    

In [ ]:
low_budget.describe()

In [ ]:
low_budget_ratings['genres'] = low_budget_ratings['genres'].str.split(',')
mid_budget_ratings['genres'] = mid_budget_ratings['genres'].str.split(',')
high_budget_ratings['genres'] = high_budget_ratings['genres'].str.split(',')

In [ ]:
low_budget_genres = low_budget_ratings.explode('genres')
mid_budget_genres = mid_budget_ratings.explode('genres')
high_budget_genres = high_budget_ratings.explode('genres')

In [ ]:
high_budget_genres.groupby('genres').agg(
  avg_wwratio=('WW_success', 'mean'),
  rng=("WW_success", lambda x: (max(x) - min(x))))

In [ ]:
high_budget_genres['genres'].value_counts()

In [ ]:
mid_budget_genres.groupby('genres').agg(
  avg_wwratio=('WW_success', 'mean'),
  rng=("WW_success", lambda x: (max(x) - min(x))))

In [ ]:
mid_budget_genres['genres'].value_counts()

In [ ]:
low_budget_genres.groupby('genres').agg(
  median_wwratio=('WW_success', 'median'),
  rng=("WW_success", lambda x: (max(x) - min(x))))

In [ ]:
low_budget_genres['genres'].value_counts()

In [ ]:
low_budget_rt_nonan = low_budget_ratings.dropna()
mid_budget_rt_nonan = mid_budget_ratings.dropna()
high_budget_rt_nonan = high_budget_ratings.dropna()

In [ ]:
rho = np.corrcoef(low_budget_rt_nonan["runtime_minutes"], low_budget_rt_nonan["WW_success"])[0][1]
s_y = low_budget_rt_nonan["WW_success"].std()
s_x = low_budget_rt_nonan["runtime_minutes"].std()
m = rho * s_y / s_x


mean_y = low_budget_rt_nonan["WW_success"].mean()
mean_x = low_budget_rt_nonan["runtime_minutes"].mean()
c = mean_y - m * mean_x
model = sm.OLS(endog=low_budget_rt_nonan["WW_success"], exog=sm.add_constant(low_budget_rt_nonan["runtime_minutes"]))
print(f"Our regression line is: y = {round(m, 5)}x + {round(c, 5)}")
results_low = model.fit()

fig, ax = plt.subplots()
low_budget_rt_nonan.plot.scatter(x="runtime_minutes", y="WW_success", label="Data points", ax=ax)
sm.graphics.abline_plot(model_results=results_low, label="Regression line", ax=ax)
ax.legend();


In [ ]:
results_low.summary()

In [ ]:
rho = np.corrcoef(mid_budget_rt_nonan["runtime_minutes"], mid_budget_rt_nonan["WW_success"])[0][1]
s_y = mid_budget_rt_nonan["WW_success"].std()
s_x = mid_budget_rt_nonan["runtime_minutes"].std()
m = rho * s_y / s_x


mean_y = mid_budget_rt_nonan["WW_success"].mean()
mean_x = mid_budget_rt_nonan["runtime_minutes"].mean()
c = mean_y - m * mean_x
model = sm.OLS(endog=mid_budget_rt_nonan["WW_success"], exog=sm.add_constant(mid_budget_rt_nonan["runtime_minutes"]))
print(f"Our regression line is: y = {round(m, 5)}x + {round(c, 5)}")
results_mid = model.fit()

fig, ax = plt.subplots()
mid_budget_rt_nonan.plot.scatter(x="runtime_minutes", y="WW_success", label="Data points", ax=ax)
sm.graphics.abline_plot(model_results=results_mid, label="Regression line", ax=ax)
ax.legend();


In [ ]:
results_mid.summary()

In [55]:
rho = np.corrcoef(high_budget_rt_nonan["runtime_minutes"], high_budget_rt_nonan["WW_success"])[0][1]
s_y = high_budget_rt_nonan["WW_success"].std()
s_x = high_budget_rt_nonan["runtime_minutes"].std()
m = rho * s_y / s_x


mean_y = high_budget_rt_nonan["WW_success"].mean()
mean_x = high_budget_rt_nonan["runtime_minutes"].mean()
c = mean_y - m * mean_x
model = sm.OLS(endog=high_budget_rt_nonan["WW_success"], exog=sm.add_constant(high_budget_rt_nonan["runtime_minutes"]))
print(f"Our regression line is: y = {round(m, 5)}x + {round(c, 5)}")
results_high = model.fit()

fig, ax = plt.subplots()
high_budget_rt_nonan.plot.scatter(x="runtime_minutes", y="WW_success", label="Data points", ax=ax)
sm.graphics.abline_plot(model_results=results_high, label="Regression line", ax=ax)
ax.legend();


NameError: name 'high_budget_rt_nonan' is not defined

In [ ]:
results_high.summary()

In [ ]:
people_money = pd.merge(people, budget_gross_cleaned, left_on = 'primary_title', right_on = 'title')

In [ ]:
people_money

In [ ]:
people_money.groupby(['primary_name', 'category']).agg(
  median_wwratio=('WW_success', 'median'),
  rng=("WW_success", lambda x: (max(x) - min(x)))).sort_values('median_wwratio', ascending = False)

In [ ]:

reg_lang_money.groupby('language').agg(
  mean_foreignratio=('foreign_success', 'mean'),
  rng=("WW_success", lambda x: (max(x) - min(x)))).sort_values('mean_foreignratio', ascending = False)

In [ ]:
reg_lang_low_budget = pd.merge(reg_lang, low_budget, left_on = 'primary_title', right_on = 'title')
reg_lang_mid_budget = pd.merge(reg_lang, mid_budget, left_on = 'primary_title', right_on = 'title')
reg_lang_high_budget = pd.merge(reg_lang, high_budget, left_on = 'primary_title', right_on = 'title')

In [ ]:
median_lowbudget_lang = reg_lang_low_budget.groupby('language').agg(
  median_foreignratio=('foreign_success', 'median'),
  rng=("foreign_success", lambda x: (max(x) - min(x)))).sort_values('median_foreignratio', ascending = False)
median_lowbudget_lang = median_lowbudget_lang.reset_index()
median_lowbudget_lang

In [ ]:
reg_lang_low_budget['language'].value_counts()

In [ ]:
low_budget_language = ['English', 'Turkish', 'Bulgarian', 'French', 'Hebrew', 'Croatian', 'Swedish']
median_lowbudget_lang = median_lowbudget_lang[median_lowbudget_lang['language'].isin(low_budget_language)]
median_lowbudget_lang

In [ ]:
median_midbudget_lang = reg_lang_mid_budget.groupby('language').agg(
  median_foreignratio=('foreign_success', 'median'),
  rng=("foreign_success", lambda x: (max(x) - min(x)))).sort_values('median_foreignratio', ascending = False)
median_midbudget_lang = median_midbudget_lang.reset_index()
median_midbudget_lang

In [ ]:
reg_lang_mid_budget['language'].value_counts()

In [ ]:
mid_budget_language = ['English', 'Turkish', 'Bulgarian', 'French', 'Hebrew', 'Croatian', 'Swedish', 'Persian', 'Catalan', 'Mandarin Chinese', 'Spanish', 'Bosnian', 'Cantonese', 'Hindi']
median_midbudget_lang = median_midbudget_lang[median_midbudget_lang['language'].isin(mid_budget_language)]
median_midbudget_lang

In [ ]:
median_highbudget_lang = reg_lang_high_budget.groupby('language').agg(
  mean_foreignratio=('foreign_success', 'mean'),
  rng=("foreign_success", lambda x: (max(x) - min(x)))).sort_values('mean_foreignratio', ascending = False)
median_highbudget_lang = median_highbudget_lang.reset_index()
median_highbudget_lang

high_budget_language = ['English', 'Turkish', 'Bulgarian', 'French', 'Hebrew', 'Croatian', 'Swedish', 'Persian', 'Catalan', 'Mandarin Chinese', 'Spanish', 'Bosnian', 'Cantonese', 'Hindi', 'Dutch', 'Tamil', 'Serbian']
median_highbudget_lang = median_highbudget_lang[median_highbudget_lang['language'].isin(high_budget_language)]
median_highbudget_lang



In [ ]:
reg_lang_high_budget['language'].value_counts()

In [ ]:
fig, ax = plt.subplots(1,3, figsize = (12,8))

ax[0].bar(median_lowbudget_lang['language'], median_lowbudget_lang['median_foreignratio'])

ax[1].bar(median_midbudget_lang['language'], median_midbudget_lang['median_foreignratio'])

ax[2].bar(median_highbudget_lang['language'], median_highbudget_lang['mean_foreignratio'])

ax[1].tick_params(axis='x', rotation=90)
ax[0].tick_params(axis='x', rotation=90)
ax[2].tick_params(axis='x', rotation=90)

ax[0].set_title('Low Budget Median Foreign Success Ration')

ax[1].set_title('Mid Budget Median Foreign Success Ration')

ax[2].set_title('High Budget Median Foreign Success Ration')

fig.tight_layout()